### 当我们在谈论prompt时我们在谈论什么

写这个编码示例的出发点是SAM的出现，SAM其实并不能算是很新的点子，其沿用了CLIP中的VIT网络架构，还有CLIP中encoder-decoder的编码，这个初衷在于不用额外的去做文本图像对齐工作，但是牺牲了一些训练时间的trade-off。SAM 同时也在CLIP的text-encoder上做了一个改进

### PointCLIP 中 prompt 的写法
pointCLIP 是用 prompt 作为一个模板

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

创建prompt模板

In [ ]:
CUSTOM_TEMPLATES = { 'ModelNet40': 'point cloud of a big {}'}

pointCLIP 中的text_encoder,实际上这个encoder还是沿用了clip的模型和文本处理器
 -> 接入CLIP的一个好选择

In [ ]:
class Textual_Encoder(nn.Module):
    def __int__(self, cfg, classnames, clip_model):
        self.cfg = cfg
        self.classnames = classnames
        self.clip_model = clip_model
        self.dtype = clip_model.dtype
        
    def forward(self):
        temp = CUSTOM_TEMPLATES[self.cfg.DATASET.NAME]
        prompts = [temp.format(c.replace('_', ' ')) for c in self.classnames ]
        prompts = torch.cat([clip.tokenize(p) for p in prompts])
        prompts = prompts.cuda()
        text_feat = self.clip_model.encode_text(prompts).repeat(1, self.cfg.MODEL.PROJECT.NUM_VIEWS)
        return text_feat

### CLIP 中 text_token的用法
首先我们从CLIP的文本结构里面看 参与作为文本编辑的